# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

импортируем библиотеки:

In [200]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

откроем файл:

In [201]:
data = pd.read_csv('/datasets/users_behavior.csv')

Выведим первые 5 строк датафрейма:

In [202]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Выведим основную информацию для датафрейма:

In [203]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Целевой признак - тариф Ультра, признаки - все столбцы, кроме Ультра

Разделим исходные данные на обучающую, валидационную и тестовую выборки:

Создадим переменные для признаков и целевого признака:

In [204]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

Выделим из исходных данных обучающую выборку:

In [205]:
data_train, data_temp = train_test_split(data, train_size=0.6, random_state=12345)

In [206]:
data.shape

(3214, 5)

In [207]:
data_train.shape

(1928, 5)

Создадим переменные для признаков и целевого признака для тренировочной выборки:

In [208]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

Оставшиеся данные поделим пополам на валидационную и тестовые выборки:

In [209]:
data_valid, data_test = train_test_split(data_temp, test_size=0.5, random_state=12345)

In [210]:
data_valid.shape

(643, 5)

In [211]:
data_test.shape

(643, 5)

Создадим переменные для признаков и целевого признака для валидационной и тестовой выборки:

In [212]:
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

## Исследуйте модели

Исследуем качество разных моделей, меняя гиперпараметры:

Исследуем качество модели по валидационной выборке:

In [213]:
for depth in range(1, 6):
    model_DTC = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model_DTC.fit(features_train, target_train)
    predictions_valid_DTC = model_DTC.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid_DTC))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


наилучший гиперпараметр примем равным: 3

In [214]:
best_model_DTC = None
best_result_DTC = 0
model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=3)
model_DTC.fit(features_train, target_train)
predictions_DTC = model_DTC.predict(features_valid)
result_DTC = accuracy_score(target_valid, predictions_DTC)
if result_DTC > best_result_DTC:
    best_model_DTC = model_DTC
    best_result_DTC = result_DTC        
"Accuracy лучшей модели(Дерево решений) валидационной выборки:", best_result_DTC

('Accuracy лучшей модели(Дерево решений) валидационной выборки:',
 0.7853810264385692)

In [215]:
for est in range(1, 11):
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators= est)
    model_RFC.fit(features_train, target_train)
    predictions_valid_RFC = model_RFC.predict(features_valid)
    print("max_est =", est, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid_RFC))

max_est = 1 : 0.7107309486780715
max_est = 2 : 0.7636080870917574
max_est = 3 : 0.7387247278382582
max_est = 4 : 0.7713841368584758
max_est = 5 : 0.749611197511664
max_est = 6 : 0.7807153965785381
max_est = 7 : 0.7682737169517885
max_est = 8 : 0.7822706065318819
max_est = 9 : 0.7729393468118196
max_est = 10 : 0.7853810264385692


наилучший гиперпараметр примем равным: 10

In [216]:
best_model_RFC = None
best_result_RFC = 0
model_RFC = RandomForestClassifier(random_state=12345, n_estimators= 10)
model_RFC.fit(features_train, target_train)
result_RFC = model_RFC.score(features_valid, target_valid)
if result_RFC > best_result_RFC:
    best_model_RFC = model_RFC
    best_result_RFC = result_RFC
print("Accuracy наилучшей модели(Случайнй лес) на валидационной выборке:", best_result_RFC)

Accuracy наилучшей модели(Случайнй лес) на валидационной выборке: 0.7853810264385692


In [217]:
best_model_LR = None
best_result_LR = 0
model_LR = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_LR.fit(features_train, target_train)
result_LR = model_LR.score(features_valid, target_valid)
if result_LR > best_result_LR:
    best_model_LR = model_LR
    best_result_LR = result_LR
print("Accuracy наилучшей модели(Логистическая регрессия) на валидационной выборке:", best_result_LR)

Accuracy наилучшей модели(Логистическая регрессия) на валидационной выборке: 0.7107309486780715


Проверим модель на тестовой выборке:

In [194]:
predictions_test = model_RFC.predict(features_test)
print("Accuracy")
print("Тестовая выборка:", accuracy_score(target_test, predictions_test) )

Accuracy
Тестовая выборка: 0.7807153965785381


In [196]:
train_predictions = model_RFC.predict(features)
test_predictions = model_RFC.predict(features_test)
print("Accuracy")
print("Обучающая выборка:", accuracy_score(target, train_predictions) )
print("Тестовая выборка:", accuracy_score(target_test, test_predictions) )

Accuracy
Обучающая выборка: 0.9026135656502801
Тестовая выборка: 0.7807153965785381


In [219]:
from sklearn.dummy import DummyClassifier
model_DC = DummyClassifier(strategy='most_frequent', random_state=1)  
model_DC.fit(features_train, target_train)
model_DC.predict(features_valid)
model_DC.score(features_test, target_test)

0.6842923794712286

На обучающей выборке модель предсказывает лучше
Качество на обучающей и тестовой выборках примерно одинаковое, высокое - мы наблюдаем эффект "недообучение"
Такой эффект наблюдается, вследствии низкого дерева обучения!